## Introduction to Quimb backend in QiboTN

#### Some imports

In [1]:
import time
import numpy as np
from scipy import stats

import qibo
from qibo import Circuit, gates, hamiltonians
from qibo.backends import construct_backend

#### Some hyper parameters

In [17]:
import cotengra as ctg
ctg_opt = ctg.ReusableHyperOptimizer(
    max_time=10,
    minimize='combo',
    slicing_opts=None,
    parallel=True,
    progbar=True
)


In [ ]:
# construct qibotn backend
quimb_backend = construct_backend(backend="qibotn", platform="quimb")

# set number of qubits
nqubits = 25

# set numpy random seed
np.random.seed(42)

quimb_backend.setup_backend_specifics(
    qimb_backend="jax", 
    optimizer='auto-hq'
    )

#### Constructing a parametric quantum circuit

In [19]:
def build_circuit(nqubits, nlayers):
    """Construct a parametric quantum circuit."""
    circ = Circuit(nqubits)
    for _ in range(nlayers):
        for q in range(nqubits):
            circ.add(gates.RY(q=q, theta=0.))
            circ.add(gates.RZ(q=q, theta=0.))
        [circ.add(gates.CNOT(q%nqubits, (q+1)%nqubits) for q in range(nqubits))]
    circ.add(gates.M(*range(nqubits)))
    return circ

In [20]:
circuit = build_circuit(nqubits=nqubits, nlayers=3)
circuit.draw()

0 :     ─RY─RZ─o───────────────────────────────────────────────X─RY─RZ─o────── ...
1 :     ─RY─RZ─X─o─────────────────────────────────────────────|─RY─RZ─X─o──── ...
2 :     ─RY─RZ───X─o───────────────────────────────────────────|─RY─RZ───X─o── ...
3 :     ─RY─RZ─────X─o─────────────────────────────────────────|─RY─RZ─────X─o ...
4 :     ─RY─RZ───────X─o───────────────────────────────────────|─RY─RZ───────X ...
5 :     ─RY─RZ─────────X─o─────────────────────────────────────|─RY─RZ──────── ...
6 :     ─RY─RZ───────────X─o───────────────────────────────────|─RY─RZ──────── ...
7 :     ─RY─RZ─────────────X─o─────────────────────────────────|─RY─RZ──────── ...
8 :     ─RY─RZ───────────────X─o───────────────────────────────|─RY─RZ──────── ...
9 :     ─RY─RZ─────────────────X─o─────────────────────────────|─RY─RZ──────── ...
10:     ─RY─RZ───────────────────X─o───────────────────────────|─RY─RZ──────── ...
11:     ─RY─RZ─────────────────────X─o─────────────────────────|─RY─RZ──────── ...
12: 

In [21]:
# Setting random parameters
circuit.set_parameters(
    parameters=np.random.uniform(-np.pi, np.pi, len(circuit.get_parameters())),
)

#### Setting up the tensor network simulator

Depending on the simulator, various parameters can be set. One can customize the tensor network execution via the `backend.configure_tn_simulation` function, whose face depends on the specific backend provider.

In [22]:
# Customization of the tensor network simulation in the case of quimb backend
# Here we use only some of the possible arguments
quimb_backend.configure_tn_simulation(
    #ansatz="MPS",
    max_bond_dimension=10
)

#### Executing through the backend

The `backend.execute_circuit` method can be used then. We can simulate results in three ways:
1. reconstruction of the final state only if `return_array` is set to `True`;
2. computation of the relevant probabilities of the final state.
3. reconstruction of the relevant state's frequencies (only if `nshots` is not `None`).

In [23]:
# Simple execution (defaults)
outcome = quimb_backend.execute_circuit(circuit=circuit, nshots=100, return_array=True)

# Print outcome
vars(outcome)

{'nqubits': 25,
 'backend': qibotn (quimb),
 'measures': Counter({'0010101100100011101100110': 1,
          '1101010011101101100010011': 1,
          '0100010101110001100001001': 1,
          '1100001000000010101011101': 1,
          '0100101000100110010010000': 1,
          '0100000111111100111001100': 1,
          '0010101100000111100000000': 1,
          '0101111100011000101100111': 1,
          '0111111001101110011000111': 1,
          '1111011100010111101010110': 1,
          '1001000111111111111000000': 1,
          '1110010011010011000010000': 1,
          '1001101011101101100000000': 1,
          '1010101100011110001110111': 1,
          '0111010110011101010101010': 1,
          '1000110100110010010011101': 1,
          '0010010001010110000011100': 1,
          '1110110010010000110100101': 1,
          '1110010110011111101110101': 1,
          '0101001000011111100010111': 1,
          '0111011101111111101101111': 1,
          '0011111111100001011000010': 1,
          '011000110

---

One can access to the specific contents of the simulation outcome.

In [26]:
print(f"Probabilities:\n {outcome.probabilities()}\n")
print(f"State:\n {outcome.state()}\n")

[Qibo 0.2.20|ERROR|2025-09-08 17:57:29]: Tensor network simulation cannot be used to reconstruct statevector for >= 20 .


Probabilities:
 {'0010101100100011101100110': np.float64(1.4287479948554065e-07), '1101010011101101100010011': np.float64(6.840535900713192e-07), '0100010101110001100001001': np.float64(2.4297656493808975e-08), '1100001000000010101011101': np.float64(8.093409548935148e-07), '0100101000100110010010000': np.float64(4.2843525768921185e-07), '0100000111111100111001100': np.float64(5.4788880664882446e-08), '0010101100000111100000000': np.float64(6.350945339007582e-09), '0101111100011000101100111': np.float64(1.6189384309546533e-09), '0111111001101110011000111': np.float64(4.231070771736262e-07), '1111011100010111101010110': np.float64(8.809840523521098e-08), '1001000111111111111000000': np.float64(2.2649917694883763e-09), '1110010011010011000010000': np.float64(2.9400450442747213e-06), '1001101011101101100000000': np.float64(3.564993949972801e-07), '1010101100011110001110111': np.float64(2.0353895972901936e-07), '0111010110011101010101010': np.float64(9.272342830457222e-08), '10001101001100

NotImplementedError: Tensor network simulation cannot be used to reconstruct statevector for >= 20 .

### Compute expectation values

Another important feature of this backend is the `expectation` function. In fact, we can compute expectation values of given observables thorugh a Qibo-friendly interface.

---

Let's start by importing some symbols, thanks to which we can build our observable.

In [27]:
# We are going to compute the expval of an Hamiltonian
# On the state prepared by the following circuit
circuit.draw()

circuit.set_parameters(
    np.random.randn(len(circuit.get_parameters()))
)

0 :     ─RY─RZ─o───────────────────────────────────────────────X─RY─RZ─o────── ...
1 :     ─RY─RZ─X─o─────────────────────────────────────────────|─RY─RZ─X─o──── ...
2 :     ─RY─RZ───X─o───────────────────────────────────────────|─RY─RZ───X─o── ...
3 :     ─RY─RZ─────X─o─────────────────────────────────────────|─RY─RZ─────X─o ...
4 :     ─RY─RZ───────X─o───────────────────────────────────────|─RY─RZ───────X ...
5 :     ─RY─RZ─────────X─o─────────────────────────────────────|─RY─RZ──────── ...
6 :     ─RY─RZ───────────X─o───────────────────────────────────|─RY─RZ──────── ...
7 :     ─RY─RZ─────────────X─o─────────────────────────────────|─RY─RZ──────── ...
8 :     ─RY─RZ───────────────X─o───────────────────────────────|─RY─RZ──────── ...
9 :     ─RY─RZ─────────────────X─o─────────────────────────────|─RY─RZ──────── ...
10:     ─RY─RZ───────────────────X─o───────────────────────────|─RY─RZ──────── ...
11:     ─RY─RZ─────────────────────X─o─────────────────────────|─RY─RZ──────── ...
12: 

In [28]:
from qibo.symbols import Z, X, I

form = 0
for i in range(nqubits):
    for _ in range(12):
        form += 0.7 * Z(i)
        form += 0.3 * X(i)

hamiltonian = hamiltonians.SymbolicHamiltonian(form)

hamiltonian.form


3.6*X0 + 3.6*X1 + 3.6*X10 + 3.6*X11 + 3.6*X12 + 3.6*X13 + 3.6*X14 + 3.6*X15 + 3.6*X16 + 3.6*X17 + 3.6*X18 + 3.6*X19 + 3.6*X2 + 3.6*X20 + 3.6*X21 + 3.6*X22 + 3.6*X23 + 3.6*X24 + 3.6*X3 + 3.6*X4 + 3.6*X5 + 3.6*X6 + 3.6*X7 + 3.6*X8 + 3.6*X9 + 8.4*Z0 + 8.4*Z1 + 8.4*Z10 + 8.4*Z11 + 8.4*Z12 + 8.4*Z13 + 8.4*Z14 + 8.4*Z15 + 8.4*Z16 + 8.4*Z17 + 8.4*Z18 + 8.4*Z19 + 8.4*Z2 + 8.4*Z20 + 8.4*Z21 + 8.4*Z22 + 8.4*Z23 + 8.4*Z24 + 8.4*Z3 + 8.4*Z4 + 8.4*Z5 + 8.4*Z6 + 8.4*Z7 + 8.4*Z8 + 8.4*Z9

In [31]:
start = time.time()
expval = quimb_backend.expectation(
    circuit=circuit,
    observable=hamiltonian,
)
elapsed = time.time() - start
print(f"Expectation value: {expval}")
print(f"Elapsed time: {elapsed:.4f} seconds")

/home/mattia/main_env/lib/python3.12/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: creg
  warnings.warn(
/home/mattia/main_env/lib/python3.12/site-packages/quimb/tensor/circuit.py:215: SyntaxWarning: Unsupported operation ignored: measure
  warnings.warn(


Expectation value: -0.9053105195111129
Elapsed time: 20.0296 seconds


Try with Qibo (which is by default using the Qibojit backend)


In [30]:
start = time.time()
result = hamiltonian.expectation(circuit().state())
elapsed = time.time() - start
print(f"Expectation value: {result}")
print(f"Elapsed time: {elapsed:.4f} seconds")

Expectation value: -0.9053096709847108
Elapsed time: 319.1915 seconds


They match! 🥳